## Notebook to Create Viz for the 2024 NCAA Frozen Four

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase
from scipy.ndimage import rotate
import os
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3  # Assuming SQL connection for database operations

from matplotlib import font_manager

## Load The Exo 2 font in case of problems with the font
font_path = 'C:/Windows/Fonts/Exo 2.tff'

# Font Location
locations =['../data/Exo_2']

font_files = font_manager.findSystemFonts(fontpaths=locations)

for file in font_files:
    font_manager.fontManager.addfont(file)



## Setup and Parameters

### Library of FF and Initial setup

In [ ]:
# FROZEN FOUR TEAMS DICTIONARY

frozen_four_teams = {
    '2013': ['Quinnipiac', 'Yale', 'UMass Lowell', 'St. Cloud State'],
    '2014': ['Minnesota', 'North Dakota', 'Union', 'Boston College'],
    '2015': ['Boston University', 'North Dakota', 'Omaha', 'Providence'],
    '2016': ['North Dakota', 'Quinnipiac', 'Denver', 'Boston College'],
    '2017': ['Denver', 'Minnesota Duluth', 'Harvard', 'Notre Dame'],
    '2018': ['Minnesota Duluth', 'Notre Dame', 'Michigan', 'Ohio State'],
    '2019': ['Massachusetts', 'Minnesota Duluth', 'Providence', 'Denver'],
    '2020': ['Minnesota Duluth', 'Massachusetts', 'North Dakota', 'Minnesota State'],
    '2021': ['Massachusetts', 'St. Cloud State', 'Minnesota State', 'Minnesota Duluth'],
    '2022': ['Michigan', 'Minnesota State', 'Minnesota', 'Denver'],
    '2023': ['Boston College', 'Michigan', 'Boston University', 'Denver']
}
# Hardcoded Frozen Four Teams for current year
teams_to_plot = ['Boston College', 'Michigan', 'Boston University', 'Denver']

## Set the column widths for the table in the scatter plots
col_widths = [0.30, 0.225, 0.225]  # Adjust the values as needed





In [ ]:
# Set Frozen Four Teams
set_region = '2023'

### OLD DEPRECATED CODE
##################################
### SET THE GLOBAL REGION
# set_region = 'Sioux Falls'
# ##################################
# ##################






### Global Settings
set_linewidth = 1
all_color='lightgrey'



import os
## File Paths
## DATA
data_folder = os.path.join(os.getcwd(), '..', '..', 'data')
NCAA_FOLDER = '../../'

## IMAGES 
# logo_folder = '../../images/logos/'  # Defined Below
background_folder = '../../images/backgrounds/'

path_to_bg_image = os.path.join(background_folder, 'main1.png')

# PAIRWISE / REGIONAL ASSIGNMENTS
pairwise_path = '../data/final_pairwise_2023.csv'


## ROSTER
# All Time Combined Roster - need to filter for 'Season'
all_time_roster_file = os.path.join(data_folder, 'rosters','all_time_combined_roster.csv')

## BOX SCORE AND ADVANCED METRICS DB
db_file = os.path.join(data_folder, 'db', 'CHN_YTD_Stats.db') # Current Year to Date Stats

# Check setup and filepaths
# all_time_roster_file, db_file

## Load Data
# Load Roster
main_roster_df = pd.read_csv(all_time_roster_file)

#LOAD PAIRWISE TABLE INTO DF
pairwise_df = pd.read_csv(pairwise_path)

# Load DB
conn = sqlite3.connect(db_file)
# Create a cursor object
cur = conn.cursor()
# Print List of Tables to check connection
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


### DEFINE TEMP FOLDER - MAKE SURE IT IS NOT OVERWRITTEN BELOW
TEMP_FOLDER = '../../NCAA/TEMP/FF/'
NCAA_FOLDER = '../../NCAA/'
HOME_FOLDER = '../../'


In [ ]:

# region_name = set_region



# # alpha_value = 0.1 # Set the transparency of the background image
# ## Load the background image
# bg_img = mpimg.imread(background_folder + path_to_bg_image)

# final_four_teams = ['Boston College', 'Michigan', 'Boston University', 'Denver']
# # Call the function with the specified parameters
# create_avg_goals_plot(region_name, conn, logo_mapping, logo_folder, final_four_teams)

### Team Color and Logo Info & Library

In [ ]:

# Team Color Information
            # path to table with hex codes
team_color_path = os.path.join('..', 'data', 'team_color_book.csv')
team_colors = pd.read_csv(team_color_path)
highlight_colors = team_colors

# Create a dictionary of team colors
team_colors_dict = pd.Series(team_colors.Hex1.values, index=team_colors.Team).to_dict()

# Create the color_mapping dictionary with the Team and Hex1 values
color_mapping = {
    
    'Boston College': team_colors_dict['Boston College'],
    'Michigan': team_colors_dict['Michigan'],
    'Boston University': team_colors_dict['Boston University'],
    'Denver': team_colors_dict['Denver'],
    }

# Logo Folder path
logo_folder = os.path.join(HOME_FOLDER, 'images', 'logos')

# Dictionary to map team names to their logo filenames
logo_mapping = {
    
    'Michigan State': 'msu.png',
    'Michigan Tech': 'mtu.png',
    'Boston College': 'bc_.png',
    'Michigan': 'mic.png',
    'Minnesota': 'min.png',
    "Wisconsin": "wis.png",
    'RIT': 'rit.png',
    'Quinnipiac': 'qui.png',
    'Cornell': 'cor.png',
    'Boston University': 'bu_.png',
    'Maine': 'mne.png',
    'Massachusetts': 'uma.png',
    'Denver': 'den.png',
    'North Dakota': 'ndk.png',
    'Omaha': 'uno.png',
    'Western Michigan': 'wmu.png'    
}

# pairwise_df.head()



## Merge PW Data with Color Data. Create dictionary of teams in each regional
# Merge pairwise with team colors
team_info_df = pairwise_df.merge(team_colors, left_on='Team', right_on='Team')
# team_info_df.head(20)






### Figure Size and Font Settings

In [ ]:
# figure_size = square_fig

#### INSTAGRAM SIZE - Portrait
dpi = 600
width, height = 1080, 1350  # in pixels
# figure_size = (width / dpi, height / dpi)  # in inches
figure_size = (10,10)

font = "Exo 2"
size1 = 24
size2 = 20
size3 = 16
# Set up Font Parameters that are used in some of the legacy code
## FONT PARAMETERS

font_title_param = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size1}
font_label_param = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size2}
font_tick_param = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size3}
font_label_large_param = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size1}
font_tick_large_param = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size1}
font_tag_parm = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size1}
font_tag_parm_2 = {'family': font, 'color': 'black', 'weight': 'normal', 'size': size1}

### Helper Functions & Logo Mapping

In [ ]:
# Filter to 2023 Season
roster_df = main_roster_df[main_roster_df['Season'] == 2023]
# Calculate average age for each team in the pairwise rankings
roster_df['DOB'] = pd.to_datetime(roster_df['DOB'])
# Calulate each players age in days
roster_df['Age'] = (pd.Timestamp.now() - roster_df['DOB']).dt.days

# Group by Team and calculate the average age for each
team_age_df = roster_df.groupby('Team')['Age'].mean().reset_index()
# Conver to years
team_age_df['Age'] = team_age_df['Age'] / 365

# roster_df.head()
# team_age_df.head()

## NEW SUB FUNCTION 3/24/24
def add_logos_to_plot(data_df, x_col, y_col, logo_mapping, logo_folder, ax, teams_in_region):
    # Plot logos for specific teams
    for team in teams_in_region:
        team_data = data_df[data_df['Team'] == team]
        if not team_data.empty:
            x = team_data[x_col].values[0]
            y = team_data[y_col].values[0]
            logo_path = os.path.join(logo_folder, logo_mapping.get(team, ""))
            if logo_path:
                try:
                    logo_img = plt.imread(logo_path)
                    imagebox = OffsetImage(logo_img, zoom=0.15)  # Adjust zoom as needed
                    ab = AnnotationBbox(imagebox, (x, y), frameon=False, xybox=(10, 20), boxcoords="offset points", pad=0)
                    ax.add_artist(ab)  # Apply the ax here as the artist is being added to ax
                except Exception as e:
                    print(f"Error adding logo for {team}: {e}")





### Check Logo Transparency - Convert to RGBA if needed

In [ ]:
import os
from PIL import Image

def convert_to_rgba(logo_mapping, logo_folder):
    for team, filename in logo_mapping.items():
        file_path = os.path.join(logo_folder, filename)
        try:
            with Image.open(file_path) as img:
                # Convert the image to RGBA if not already in that mode
                if img.mode != 'RGBA':
                    rgba_image = img.convert('RGBA')
                    # Save the converted image back to the folder
                    rgba_image.save(file_path)
                    print(f"Converted {filename} to RGBA.")
                else:
                    print(f"{filename} is already RGBA.")
        except FileNotFoundError:
            print(f"File {filename} not found in {logo_folder}.")
        except Exception as e:
            print(f"An error occurred with {filename}: {e}")

# Run the conversion function
convert_to_rgba(logo_mapping, logo_folder)

# Function to check if images have an alpha channel
def check_image_channels(logo_mapping, logo_folder):
    results = {}
    for team, filename in logo_mapping.items():
        try:
            file_path = os.path.join(logo_folder, filename)
            with Image.open(file_path) as img:
                # Check if the image has an alpha channel
                results[team] = "Alpha channel present" if img.mode == 'RGBA' else "No alpha channel"
        except FileNotFoundError:
            results[team] = "File not found"
        except Exception as e:
            results[team] = str(e)
    return results

# Run the check and print the results
alpha_check_results = check_image_channels(logo_mapping, logo_folder)
alpha_check_results



In [ ]:
roster_df = main_roster_df[main_roster_df['Season'] == 2023]
# Rename Clean_name to Player
roster_df = roster_df.rename(columns={'Clean_Name': 'Player'})
# roster_df.head()

## Start Plotting

## Goals by Game Time



In [ ]:
#### Refactoring from ORIG code in Adaptable Plotting Book
import matplotlib.ticker as ticker

## Functions
### get the basic info from the scoring summary table
def fetch_goals_time_data(conn):
    goals_time_query = """
    WITH UniqueGoals AS (
        SELECT DISTINCT Team, Period, Time, Game_ID
        FROM scoring_summary
    )
    SELECT Team, Period, Time, Game_ID
    FROM UniqueGoals;
    """
    return pd.read_sql(goals_time_query, conn)


### Convert the time to continuous time [0-65] for plotting
def convert_to_continuous_time(row):
    period = row['Period']
    time = row['Time']
    minutes, seconds = map(int, time.split(':'))
    
    if period == '1st Period':
        offset = 0
    elif period == '2nd Period':
        offset = 20
    elif period == '3rd Period':
        offset = 40
    else:
        offset = 60
        
    return offset + minutes + seconds / 60.0

goals_time_df = fetch_goals_time_data(conn)

goals_time_df['Time'] = goals_time_df.apply(convert_to_continuous_time, axis=1)

# goals_time_df.head()



In [ ]:
# Revised function to correctly parse team names, accounting for additional hyphens in team names
def infer_conceded_goals_corrected_v4(df):
    conceded_goals = []
    for index, row in df.iterrows():
        # Split the game ID to find the teams involved
        # Since teams can have hyphens in their names, we take all parts after the date
        game_id_parts = row['Game_ID'].split('-')
        teams_in_game = ['-'.join(game_id_parts[3:4]), '-'.join(game_id_parts[4:])]  # Correctly extract the team names
        # Identify the opposing team
        opposing_team = teams_in_game[1] if row['Team'] == teams_in_game[0] else teams_in_game[0]
        # Add a row for the conceded goal
        conceded_goals.append({'Team': opposing_team, 'Time': row['Time'], 'Game_ID': row['Game_ID']})
    
    # Create a DataFrame for conceded goals
    conceded_goals_df = pd.DataFrame(conceded_goals)
    return conceded_goals_df

# Create the corrected conceded goals dataframe
conceded_goals_df_corrected_v4 = infer_conceded_goals_corrected_v4(goals_time_df)

# Display the first few rows of the corrected conceded goals dataframe
# conceded_goals_df_corrected_v4.head(25)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np


def plot_goals_by_time(region_name, goals_time_df, conceded_goals_df_corrected_v4, region_dict, TEMP_FOLDER, dpi):
    half_page = (10,10)  # Half Page
    fig, ax = plt.subplots(4, 1, figsize=figure_size, dpi=dpi)

    

    # Calculate the max y-value and histograms for all teams
    max_y_value = 0
    team_histograms = {}
    for team in teams_to_plot:
        team_goals_time_df = goals_time_df[goals_time_df['Team'] == team]
        bin_counts, _ = np.histogram(team_goals_time_df['Time'], bins=75, range=[0, 75])
        max_y_value = max(max_y_value, max(bin_counts))
        team_histograms[team] = bin_counts


    ## Loop through each team and plot the histogram of goals scored by time
    for i, team in enumerate(teams_to_plot):
        team_goals_time_df = goals_time_df[goals_time_df['Team'] == team]
        
        # Retrieve the team color from the dictionary
        team_color = team_colors_dict.get(team, 'default_color')  # Provide a default color if the team is not found

        # Plot histogram
        sns.histplot(data=team_goals_time_df, x='Time', bins=75, ax=ax[i], color=team_color, zorder=1)
        
        # # Plot KDE with adjusted bandwidth
        sns.kdeplot(data=team_goals_time_df, x='Time', ax=ax[i], color="red", bw_adjust=0.5, zorder=2)

        # Plot the goals conceded by the team as a histogram in the background in Gray
        team_conceded_goals_df = conceded_goals_df_corrected_v4[conceded_goals_df_corrected_v4['Team'] == team]
        sns.histplot(data=team_conceded_goals_df, x='Time', bins=75, ax=ax[i], color='grey', zorder=2, linewidth=0.5, edgecolor='grey', alpha=0.5)


        
        # Set the x and y axis limits
        ax[i].set_xlim([0, 75])
        ax[i].set_ylim([0, max_y_value+1])

        # Custom x-axis ticks and labels for the periods
        period_ticks = [10, 30, 50, 67.5]  # Midpoints of 1st, 2nd, 3rd periods and OT
        period_labels = ['1st Period', '2nd Period', '3rd Period', 'OT']
        ax[i].set_xticks(period_ticks)
        ax[i].set_xticklabels(period_labels, fontsize=12)

        # Add Major grid lines at 20, 40, 60 minute marks to deliniate periods
        
        

        
        # Set the x axis font properties
        ax[i].tick_params(axis='x', which='major', labelsize=12)

        # Remove x label
        ax[i].set_xlabel('')
        # remove tick marks from x axis
        ax[i].tick_params(axis='x', which='major', length=0)
        # Set y label
        ax[i].set_ylabel(f'{team}', fontsize=14)
        # remove tick marks from y axis
        ax[i].tick_params(axis='y', which='major', length=0)
        # remove y tick labels
        # ax[i].set_yticklabels([])
        

        # Set grid, background color, and border properties
        ax[i].grid(True, axis='y', color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.9, zorder=0)
        ax[i].grid(True, axis='x', color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.9, zorder=0)
        # shade the periods - first and third periods (0 to 20 and 40 to 60) in light grey
        ax[i].axvspan(0, 20, facecolor='lightgrey', alpha=0.5, zorder=0)
        ax[i].axvspan(40, 60, facecolor='lightgrey', alpha=0.5, zorder=0)
        # Shade 2nd period and OT in light yeallow
        ax[i].axvspan(20, 40, facecolor='lightyellow', alpha=0.75, zorder=0)
        ax[i].axvspan(60, 75, facecolor='lightyellow', alpha=0.75, zorder=0)

        # Remove The x labels from all but the last plot
        # if i != 3:
        #     ax[i].set_xlabel('')
        #     ax[i].set_xticklabels([])

        # Add the x ticks and labels but only to the fisrt plot and above the plot
        if i == 0:
            ax[i].tick_params(axis='x', which='major', pad=10, top=True, labeltop=True)
            # Remove tick marks from bottom of plot
            ax[i].tick_params(axis='x', which='major', bottom=False, labelbottom=False)
            # ax[i].set_xlabel('Game Time', font_label_param)

        else:        
            # Remove tick labels from all but plot 1
            ax[i].set_xticklabels([])

    # Adjust the spacing between plots
    fig.tight_layout()


    # add the tick labels to the right side of the plot
    ax[0].tick_params(axis='y', labelleft=False, labelright=True)
    ax[1].tick_params(axis='y', labelleft=False, labelright=True)
    ax[2].tick_params(axis='y', labelleft=False, labelright=True)
    ax[3].tick_params(axis='y', labelleft=False, labelright=True)


    # Label just 3 ticks on the y axis, the minium, the midpoint and the maximum
    ax[0].set_yticks([0, 5])
    ax[1].set_yticks([0, 5])
    ax[2].set_yticks([0, 5])
    ax[3].set_yticks([0, 5])

    # Add Minor grid lines
    ax[0].minorticks_on()  # Enable minor ticks
    ax[0].grid(color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.6, which='minor', axis='y')
    ax[1].minorticks_on()  # Enable minor ticks
    ax[1].grid(color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.6, which='minor', axis='y')
    ax[2].minorticks_on()  # Enable minor ticks
    ax[2].grid(color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.6, which='minor', axis='y')
    ax[3].minorticks_on()  # Enable minor ticks
    ax[3].grid(color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.6, which='minor', axis='y')

    # Change all gird lines to underneath plot elements
    for i in range(4):
        ax[i].set_axisbelow(True)

        # Set title for entire figure
    fig.suptitle(f'Goals vs Game Time: 2024 Frozen Four', fontsize=font_title_param['size'], fontfamily=font_title_param['family'], fontweight=font_title_param['weight'], color=font_title_param['color'])

    # Add Text box with note 'Team Color - Goals Scored, Gray - Goals Conceded'
    fig.text(0.5, 0.01, 'Team Color: Goals Scored | Gray: Goals Conceded | Each Bar is Equal to 1 Minute Increment', ha='center', va='center', fontsize=14, color='black')

    fig.tight_layout()

    # Set background to white
    fig.patch.set_facecolor('white')

    # Save the plot
    plt.savefig(f'{TEMP_FOLDER}{region_name}-goals_scored_by_time.png', bbox_inches='tight', dpi=dpi, transparent=False)

    # Show the plot
    plt.show()


In [ ]:
# Call Function
plot_goals_by_time(set_region, goals_time_df, conceded_goals_df_corrected_v4, frozen_four_teams, TEMP_FOLDER, dpi)

## Age And Experience

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import os

###### SETTINGS ######
zoom_factor = 0.25
y_offset = 1.1
age_offset = -.125
age_fontsize = 12
figure_size = (10, 10)


def plot_team_ages_and_experience_with_logos(roster_df, team_names, region_name, logo_mapping, logo_folder, pairwise_df, figure_size=(10, 10)):
    """
    Plots a stacked bar chart comparing the age and experience of hockey teams, using team logos for x-axis labels.
    
    Parameters:
    - roster_df: DataFrame containing the team rosters.
    - team_names: List of the team names to be included in the plot.
    - region_name: The name of the region for the title of the plot.
    - logo_mapping: Dictionary mapping team names to logo image filenames.
    - logo_folder: Path to the folder containing the logo images.
    - figure_size: Tuple indicating the figure size.
    """
    
    # Process data as before
    filtered_df = roster_df[roster_df['Team'].isin(team_names)]
    # Lock In the Order of teams in the 
    desired_order = ['Boston College', 'Michigan', 'Boston University', 'Denver'] # Lock In the Order of teams in the plot
    # Filter dataframe: contains only teams set int he 
    filtered_df = roster_df[roster_df['Team'].isin(desired_order)]
    correct_order = ['Fr', 'So', 'Jr', 'Sr', 'Gr'] # set order of class rank
    
    # Create a categorical type for 'Team' based on the desired order to ensure plots follow this order
    filtered_df['Team'] = pd.Categorical(filtered_df['Team'], categories=desired_order, ordered=True)

    # Reverse the order of the ranks
    # correct_order.reverse()
    class_counts = filtered_df.groupby(['Team', 'Yr'])['Player'].count().unstack().fillna(0)
    class_proportions = class_counts.div(class_counts.sum(axis=1), axis=0)
    
    available_ranks = [rank for rank in correct_order if rank in class_proportions.columns]
    class_proportions_corrected = class_proportions[available_ranks]
    
    # Merge the class proportions with pairwise_df to get the ranks
    class_proportions_corrected_with_rank = class_proportions_corrected.merge(pairwise_df, left_on='Team', right_on='Team')
    
    # Sort the dataframe by rank
    class_proportions_sorted = class_proportions_corrected_with_rank # .sort_values('Rk')
    
    # Update team_names list based on sorted order
    team_names_sorted = class_proportions_sorted['Team'].tolist()

    # Create the plot with the sorted dataframe
    fig, ax = plt.subplots(figsize=figure_size)
    bars = class_proportions_sorted[available_ranks].plot(kind='bar', stacked=True, ax=ax, colormap='Set3')
    ax.set_title(f'Age and Experience Comparison\n  2024 Frozen Four')
    ax.set_xlabel('Teams')
    ax.set_ylabel('Proportion of Players (%)')

    # # ORIGINAL LEGEND CODE
    ax.legend(title='Class Rank', fontsize=16).set_draggable(True)
    # Reverse the order in the legend to match the bars
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(reversed(handles), reversed(labels), title='Class Rank', fontsize=16, loc='lower center').set_draggable(True)

    # Create a custom legend
    # Reverse the order in the legend to match the bars
    # handles, labels = ax.get_legend_handles_labels()
    # # Create the legend with a horizontal layout (ncol=len(handles)) and place it on top of the plot under the title
    # # Adjust the 'bbox_to_anchor' values to position the legend exactly where you want it
    # ax.legend(reversed(handles), reversed(labels), title='Class Rank', fontsize=16, loc='upper center', ncol=len(handles), frameon=False, bbox_to_anchor=(0.5, 1.05))


    # Iterate through the patches (bars) to add the text annotations
    for bar in bars.patches:
        # Get the bar's height, width, and position
        height = bar.get_height()
        width = bar.get_width()
        x, y = bar.get_xy()
        # The label is set to be at 50% of the height of the bar, you can adjust it if necessary
        label_text = f'{height:.1%}'
        label_x = x + width / 2
        label_y = y + height / 2
        
        # Only add text if the bar is tall enough to fit the text
        if height > 0.05:  # 5% threshold, can be adjusted
            ax.text(label_x, label_y, label_text, ha='center', va='center', color='black', fontsize=12)


    
    ######### AXIS AND LABEL SETTINGS #########
    # Adjust the Title Properties
    ax.title.set_fontsize(font_title_param['size'])
    # Set the x-axis Title to blank
    ax.set_xlabel('')
    # Set y-axis label to blank
    # ax.set_ylabel('')
    ax.set_ylabel('Proportion of Players (%)', fontdict=font_label_large_param)
    # Remove x-tick labels
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Add logos as x-tick labels
    for i, team in enumerate(team_names):
        if team in logo_mapping:
            file_path = os.path.join(logo_folder, logo_mapping[team])
            img = plt.imread(file_path)
            imagebox = OffsetImage(img, zoom=zoom_factor)
            ab = AnnotationBbox(imagebox, (i, 0), xycoords=('data', 'axes fraction'),
                                boxcoords="offset points", box_alignment=(0.5, y_offset), pad=0, frameon=False)
            ax.add_artist(ab)
    
        # Retrieve the average age for the team
        team_age = team_age_df.loc[team_age_df['Team'] == team, 'Age'].values[0]
        
        team_age_text = f'Avg Age {team_age:.2f}'  # Add "Avg Age" prefix and format the age with 2 decimal places

        # Define the x and y position for the team age text
        age_text_x = i
        age_text_y = y_offset + age_offset  # You can adjust age_offset to move the text up or down

        # Add the age text to the plot
        ax.text(age_text_x, age_text_y, team_age_text, ha='center', va='center', fontsize=age_fontsize, weight='bold', 
                transform=ax.get_xaxis_transform())

    # Save the plot to TEMP_FOLDER
    plot_path = f'{TEMP_FOLDER}{region_name}_age_experience_plot.png'
    plt.savefig(plot_path, bbox_inches='tight')
        

    plt.show()

region_def = set_region


In [ ]:
### WRONG ORDER OF TEAMS - NEED TO FIX

# Example usage
plot_team_ages_and_experience_with_logos(roster_df, frozen_four_teams[region_def], region_def, logo_mapping, logo_folder, pairwise_df, figure_size=figure_size)

## Scatter Plots

## Shots

### Code

In [ ]:
def create_avg_shots_plot(region_name, conn, logo_mapping, logo_folder, frozen_four_teams):
    figure_size = (10, 10)  # Optimal size for platforms like Instagram

    avg_shots_query = """
    WITH UniqueGames AS (
        SELECT DISTINCT * FROM linescore
    )
    SELECT
        a.Team,
        AVG(a.shotsT) AS Avg_Shots_Taken,
        AVG(b.shotsT) AS Avg_Shots_Allowed
    FROM UniqueGames AS a
    JOIN UniqueGames AS b ON a.Game_ID = b.Game_ID AND a.Team != b.Team
    GROUP BY a.Team;

    """
    
    # Execute the query to fetch data for all teams
    avg_shots_df = pd.read_sql(avg_shots_query, conn)

    # Calculate average and standard deviation for "Shots Taken" and "Shots Allowed"
    avg_shots_taken = avg_shots_df['Avg_Shots_Taken'].mean()
    std_shots_taken = avg_shots_df['Avg_Shots_Taken'].std()
    avg_shots_allowed = avg_shots_df['Avg_Shots_Allowed'].mean()
    std_shots_allowed = avg_shots_df['Avg_Shots_Allowed'].std()

    # Create the scatter plot for all teams
    fig, ax = plt.subplots(figsize=figure_size)
    sns.scatterplot(data=avg_shots_df, x='Avg_Shots_Allowed', y='Avg_Shots_Taken', color='darkgrey', ax=ax)

    # Add trend line
    sns.regplot(data=avg_shots_df, x='Avg_Shots_Allowed', y='Avg_Shots_Taken', scatter=False, color='blue')

    # Adjust x-axis to add padding on the left
    xmin, xmax = ax.get_xlim()
    new_xmin = xmin  # Subtract 2 from the x-minimum for additional padding
    ax.set_xlim(new_xmin, xmax)

    # Get and Adjust y-limits if needed
    ymin, ymax = ax.get_ylim()

    # Now, plot the average lines and standard deviation markers after setting the axis limits
    plt.axhline(y=avg_shots_taken, color='red', linestyle='--', linewidth=0.8)
    plt.axvline(x=avg_shots_allowed, color='red', linestyle='--', linewidth=0.8)
    plt.errorbar(avg_shots_allowed, avg_shots_taken, xerr=std_shots_allowed, 
                 yerr=std_shots_taken, color='purple', fmt='o')

    # Set the background image
    # Ensure that the extent of the background image is also adjusted to the new xmin
    plt.imshow(bg_img, aspect='auto', extent=[new_xmin, xmax, ymin, ymax], zorder=0, alpha=alpha_value)

    # Overlay logos for specific teams in the region
    teams_in_region = frozen_four_teams[region_name]
    add_logos_to_plot(avg_shots_df, 'Avg_Shots_Allowed', 'Avg_Shots_Taken', logo_mapping, logo_folder, ax, teams_in_region)

    # Create table data
    table_data = [['Team', 'Avg Shots', 'Avg Allowed']]
    for team in teams_in_region:
        team_data = avg_shots_df[avg_shots_df['Team'] == team]
        if not team_data.empty:
            table_data.append([
                team, 
                f"{team_data['Avg_Shots_Taken'].values[0]:.2f}",
                f"{team_data['Avg_Shots_Allowed'].values[0]:.2f}"
            ])

    # Define the column widths, making the first column wider
    # col_widths = [0.25, 0.225, 0.225]  # Adjust the values as needed

    # Add a table at the top right of the plot & Set Parameters
    table = plt.table(cellText=table_data, loc='best', 
                      colWidths=col_widths,
                      cellLoc='center', 
                      bbox=[0.45, 0, 0.5, 0.3]
                      )
    
    # Customize the table properties
    table.auto_set_font_size(False)
    table.set_fontsize(12)  # Set the fontsize
    table.scale(1, 1.5)  # Scale the table for width and height

    # Set individual cell properties
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            # Header row
            cell.set_text_props(weight='bold')  # Make the header bold
            cell.set_facecolor('lightgrey')  # Header background color
        else:
            # All other rows
            cell.set_facecolor('none')  # Make cells transparent
            # cell.set_text_props(weight='bold')  # Optionally make text bold
        cell.set_edgecolor('w')  # Set the border color to white

    # Add title and axis labels
    plt.title(f'Shots Per Game: 2024 Frozen Four', fontdict=font_title_param)
    plt.xlabel('Average Shots Allowed', fontdict=font_label_param)
    plt.ylabel('Average Shots Taken', fontdict=font_label_param)


    # Save plot to TEMP_FOLDER
    plot_path = f'{TEMP_FOLDER}{region_name}_shots_plot.png'
    plt.savefig(plot_path, bbox_inches='tight')


    plt.show()





### Output

In [ ]:

region_name = set_region


# #### Set Background Image
path_to_bg_image =  'main1.png' #
alpha_value = 0.1 # Set the transparency of the background image
## Load the background image
bg_img = mpimg.imread(background_folder + path_to_bg_image)

# Call the function with the specified parameters for 2023 final four teams
create_avg_shots_plot(region_name, conn, logo_mapping, logo_folder, frozen_four_teams)

## Goals Per Game

### Code

In [ ]:
def create_avg_goals_plot(region_name, conn, logo_mapping, logo_folder, region_dict):
    figure_size = (10, 10)  # Optimal size for platforms like Instagram
    
    ## New Query to calvulate average goals using the player stats ytd table
    ## Sum Goals by team count the instances of Unique Game IDin game details for each team
    avg_goals_query = """
    WITH UniqueGames AS (
        SELECT DISTINCT * FROM linescore
    )

    SELECT 
        UG1.Team, 
        SUM(UG1.goalsT) AS Total_Goals_Scored, 
        SUM(UG2.goalsT) AS Total_Goals_Allowed,
        COUNT(DISTINCT UG1.Game_ID) AS Games_Played
    FROM 
        UniqueGames UG1
    JOIN 
        UniqueGames UG2 ON UG1.Game_ID = UG2.Game_ID AND UG1.Team != UG2.Team
    GROUP BY 
        UG1.Team;
    """



    # Execute the query and store the results in a DataFrame
    avg_goals_df = pd.read_sql(avg_goals_query, conn)
    final_four_teams = ['Boston College', 'Michigan', 'Boston University', 'Denver']
    # avg_goals_df = avg_goals_df[avg_goals_df['Team'].isin(final_four_teams)]

    # Calculate averages of goals scored and allowed
    avg_goals_df['Avg_Goals_Scored'] = avg_goals_df['Total_Goals_Scored'] / avg_goals_df['Games_Played']
    avg_goals_df['Avg_Goals_Allowed'] = avg_goals_df['Total_Goals_Allowed'] / avg_goals_df['Games_Played']


    # Calculate average and standard deviation for "Goals Scored" and "Goals Allowed"
    avg_goals_scored = avg_goals_df['Avg_Goals_Scored'].mean()
    std_goals_scored = avg_goals_df['Avg_Goals_Scored'].std()
    avg_goals_allowed = avg_goals_df['Avg_Goals_Allowed'].mean()
    std_goals_allowed = avg_goals_df['Avg_Goals_Allowed'].std()
    
    # # Create the scatter plot for all teams
    fig, ax = plt.subplots(figsize=figure_size)
    sns.scatterplot(data=avg_goals_df, x='Avg_Goals_Allowed', y='Avg_Goals_Scored', color='darkgrey', ax=ax)

    # Add trend line
    sns.regplot(data=avg_goals_df, x='Avg_Goals_Allowed', y='Avg_Goals_Scored', scatter=False, color='blue')

    # Adjust x-axis to add padding on the left
    xmin, xmax = ax.get_xlim()
    new_xmin = xmin  # Legacy
    ax.set_xlim(new_xmin, xmax)

    # Get and Adjust y-limits if needed
    ymin, ymax = ax.get_ylim()
    new_ymax = ymax + 0.25

    # Now, plot the average lines and standard deviation markers after setting the axis limits
    plt.axhline(y=avg_goals_scored, color='red', linestyle='--', linewidth=0.8)
    plt.axvline(x=avg_goals_allowed, color='red', linestyle='--', linewidth=0.8)
    plt.errorbar(avg_goals_allowed, avg_goals_scored, xerr=std_goals_allowed, 
                 yerr=std_goals_scored, color='purple', fmt='o')

    # Set the background image
    # Ensure that the extent of the background image is also adjusted to the new xmin
    plt.imshow(bg_img, aspect='auto', extent=[new_xmin, xmax, ymin, new_ymax], zorder=0, alpha=alpha_value)

    # Overlay logos for specific teams in the region
    teams_in_region = final_four_teams
    add_logos_to_plot(avg_goals_df, 'Avg_Goals_Allowed', 'Avg_Goals_Scored', logo_mapping, logo_folder, ax, teams_in_region)

    # Create table data
    table_data = [['Team', 'Goals For', 'Goals Against']]
    for team in teams_in_region:
        team_data = avg_goals_df[avg_goals_df['Team'] == team]
        if not team_data.empty:
            table_data.append([
                team, 
                f"{team_data['Avg_Goals_Scored'].values[0]:.2f}",
                f"{team_data['Avg_Goals_Allowed'].values[0]:.2f}"
            ])

    # Define the column widths, making the first column wider
    # col_widths = [0.30, 0.225, 0.225]  # Adjust the values as needed

    # Add a table at the top right of the plot & Set Parameters
    table = plt.table(cellText=table_data, loc='best', 
                      colWidths=col_widths,
                      cellLoc='center', 
                      bbox=[0.45, 0, 0.5, 0.3]
                      )
    
    # Customize the table properties
    table.auto_set_font_size(False)
    table.set_fontsize(12)  # Set the fontsize
    table.scale(1, 1.5)  # Scale the table for width and height

    # Set individual cell properties
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            # Header row
            cell.set_text_props(weight='bold')  # Make the header bold
            cell.set_facecolor('lightgrey')  # Header background color
        else:
            # All other rows
            cell.set_facecolor('none')  # Make cells transparent
            # cell.set_text_props(weight='bold')  # Optionally make text bold
        cell.set_edgecolor('w')  # Set the border color to white

        # Add title and axis labels
        plt.title(f'Goals Per Game: 2024 Frozen Four', fontdict=font_title_param)
        plt.xlabel('Average Goals Allowed', fontdict=font_label_large_param)
        plt.ylabel('Average Goals Scored', fontdict=font_label_large_param)


    # Save plot To TEMP_FOLDER
    plot_path = f'{TEMP_FOLDER}{region_name}_goals_plot.png'
    
    # Save
    plt.savefig(plot_path, bbox_inches='tight')


    plt.show()





### Output

In [ ]:

region_name = set_region



# alpha_value = 0.1 # Set the transparency of the background image
## Load the background image
bg_img = mpimg.imread(background_folder + path_to_bg_image)

final_four_teams = ['Boston College', 'Michigan', 'Boston University', 'Denver']
# Call the function with the specified parameters
create_avg_goals_plot(region_name, conn, logo_mapping, logo_folder, final_four_teams)

## Penalty Minutes

### Code

In [ ]:
def create_avg_penalty_plot(region_name, conn, logo_mapping, logo_folder, region_dict):
    # SQL query to calculate the average penalty minutes "for" and "against" each team
    avg_penalty_for_against_query = """
    WITH UniqueGames AS (
    SELECT DISTINCT Team, Game_ID, PIM FROM linescore
    )
    SELECT
        a.Team,
        AVG(a.PIM) AS Avg_Penalty_Minutes_For,
        AVG(b.PIM) AS Avg_Penalty_Minutes_Against
    FROM UniqueGames AS a
    JOIN UniqueGames AS b ON a.Game_ID = b.Game_ID AND a.Team != b.Team
    GROUP BY a.Team;
    """

    
    
    
    ### OLD CODE - ERROR PRONE
    # avg_penalty_for_against_query = """
    # WITH UniquePenalties AS (
    #     SELECT DISTINCT * FROM penalty_summary
    # ),
    # PenaltyFor AS (
    #     SELECT
    #         Team,
    #         AVG(Total_Penalty_Minutes) AS Avg_Penalty_Minutes_For
    #     FROM (
    #         SELECT
    #             Team,
    #             Game_ID,
    #             SUM(Pen_Length) AS Total_Penalty_Minutes
    #         FROM UniquePenalties
    #         GROUP BY Team, Game_ID
    #     )
    #     GROUP BY Team
    # ),
    # PenaltyAgainst AS (
    #     SELECT
    #         b.Team,
    #         AVG(a.Total_Penalty_Minutes) AS Avg_Penalty_Minutes_Against
    #     FROM (
    #         SELECT
    #             Team,
    #             Game_ID,
    #             SUM(Pen_Length) AS Total_Penalty_Minutes
    #         FROM UniquePenalties
    #         GROUP BY Team, Game_ID
    #     ) AS a
    #     JOIN UniquePenalties AS b ON a.Game_ID = b.Game_ID AND a.Team != b.Team
    #     GROUP BY b.Team
    # )
    # SELECT
    #     a.Team,
    #     a.Avg_Penalty_Minutes_For,
    #     b.Avg_Penalty_Minutes_Against
    # FROM PenaltyFor AS a
    # JOIN PenaltyAgainst AS b ON a.Team = b.Team;

    # """

    # Execute the query and store the results in a DataFrame
    avg_penalty_for_against_df = pd.read_sql(avg_penalty_for_against_query, conn)
    # avg_penalty_for_against_df.head()

    # Calculate average and standard deviation for "For" and "Against"
    avg_for = avg_penalty_for_against_df['Avg_Penalty_Minutes_For'].mean()
    std_for = avg_penalty_for_against_df['Avg_Penalty_Minutes_For'].std()
    avg_against = avg_penalty_for_against_df['Avg_Penalty_Minutes_Against'].mean()
    std_against = avg_penalty_for_against_df['Avg_Penalty_Minutes_Against'].std()

    figure_size = (10,10)
    # Create the scatter plot for all teams
    fig, ax = plt.subplots(figsize=figure_size)
    sns.scatterplot(data=avg_penalty_for_against_df, x='Avg_Penalty_Minutes_Against', y='Avg_Penalty_Minutes_For', color='darkgrey', ax=ax)

    # get and store the x and y limits
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    # Add trend line
    sns.regplot(data=avg_penalty_for_against_df, x='Avg_Penalty_Minutes_Against', y='Avg_Penalty_Minutes_For', scatter=False, color='blue')

    # Add average lines and standard deviation markers
    plt.axhline(y=avg_for, color='red', linestyle='--', linewidth=0.8)
    plt.axvline(x=avg_against, color='red', linestyle='--', linewidth=0.8)
    plt.errorbar(avg_against, avg_for, xerr=std_against, yerr=std_for, color='purple', fmt='o')

    # Set the background image
    # Ensure that the extent of the background image is also adjusted to the new xmin
    plt.imshow(bg_img, aspect='auto', extent=[xmin, xmax, ymin, ymax], zorder=0, alpha=alpha_value)

    # Overlay logos for specific teams in the region
    teams_in_region = final_four_teams

    add_logos_to_plot(avg_penalty_for_against_df, 'Avg_Penalty_Minutes_Against', 'Avg_Penalty_Minutes_For', logo_mapping, logo_folder, ax, teams_in_region)

    # Create table data
    table_data = [['Team', 'PIM', 'Opp PIM']]
    for team in teams_in_region:
        team_data = avg_penalty_for_against_df[avg_penalty_for_against_df['Team'] == team]
        if not team_data.empty:
            table_data.append([
                team, 
                f"{team_data['Avg_Penalty_Minutes_For'].values[0]:.1f}",
                f"{team_data['Avg_Penalty_Minutes_Against'].values[0]:.1f}"
            ])

    # Define the column widths, making the first column wider
    # col_widths = [0.25, 0.225, 0.225]  # SET GLOBALLY

    # Add a table at the top right of the plot & Set Parameters
    table = plt.table(cellText=table_data, loc='best', 
                      colWidths=col_widths,
                      cellLoc='center', 
                      bbox=[0.45, 0, 0.5, 0.3]
                      )
    
    # Customize the table properties
    table.auto_set_font_size(False)
    table.set_fontsize(12)  # Set the fontsize
    table.scale(1, 1.5)  # Scale the table for width and height

    # Set individual cell properties
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            # Header row
            cell.set_text_props(weight='bold')  # Make the header bold
            cell.set_facecolor('lightgrey')  # Header background color
        else:
            # All other rows
            cell.set_facecolor('none')  # Make cells transparent
            # cell.set_text_props(weight='bold')  # Optionally make text bold
        cell.set_edgecolor('w')  # Set the border color to white

    # Add title and axis labels
    plt.title(f'Penalty Minutes Per Game: 2024 Frozen Four', fontdict=font_title_param)
    plt.xlabel('Opponent Penalty Minutes', fontdict=font_label_param)
    plt.ylabel('Team Penalty Minutes', fontdict=font_label_param)

    # SAVE PLOT TO TEMP_FOLDER
    plot_path = f'{TEMP_FOLDER}{region_name}_penalty_plot.png'
    plt.savefig(plot_path, bbox_inches='tight')

    plt.show()





### Output

In [ ]:

# Set Region Name
region_name = set_region

# Call the function with the specified parameters
create_avg_penalty_plot(region_name, conn, logo_mapping, logo_folder, final_four_teams)

# Penalties over Time

## Code

In [ ]:
### HOTFIX
# use the color mapping dictionary to create the color_map nessisaary for the plot
color_mapping = {
    'Boston College': 'maroon',
    'Michigan': 'blue',
    'Boston University': 'red',
    'Denver': 'crimson'
}

# select the penalty summary table
penalty_summary_query = """
WITH UniquePenalties AS (
    SELECT DISTINCT * FROM penalty_summary
)
SELECT * FROM UniquePenalties;
"""

# Read the penalty summary table into a dataframe
penalty_summary_df = pd.read_sql(penalty_summary_query, conn)

# output to csv in temp folder
penalty_summary_df.to_csv(f'{TEMP_FOLDER}penalty_summary.csv', index=False)


# Penalt Length to int
penalty_summary_df['Pen_Length'] = penalty_summary_df['Pen_Length'].astype('int')



### Convert the time to continuous time [0-65] for plotting
def convert_to_continuous_time(row):
    period = row['Period']
    time = row['Time']
    minutes, seconds = map(int, time.split(':'))
    
    if period == '1st Period':
        offset = 0
    elif period == '2nd Period':
        offset = 20
    elif period == '3rd Period':
        offset = 40
    else:
        offset = 60
        
    return offset + minutes + seconds / 60.0



def plot_team_penalties(team, penalty_data, color_mapping=None, ax=None, smoothing_window=5, bins=65):
    # Convert times to continuous format
    penalty_data['Start_Time'] = penalty_data.apply(convert_to_continuous_time, axis=1)
    penalty_data['End_Time'] = penalty_data['Start_Time'] + penalty_data['Pen_Length']

    # Filter out 10-minute penalties
    penalty_data = penalty_data[penalty_data['Pen_Length'].isin([2, 5])]

    # Identify Game IDs involving the specified team
    team_game_ids = penalty_data[penalty_data['Team'] == team]['Game_ID'].unique()

    # Filter penalties to those in the team's games
    team_games_penalty_data = penalty_data[penalty_data['Game_ID'].isin(team_game_ids)]

    # Initialize time series
    time_points = np.linspace(0, 65, 65 * 60)  # 1-second intervals
    team_penalty_count = np.zeros_like(time_points)
    opponent_penalty_count = np.zeros_like(time_points)

    # Calculate penalty count for team and opponents
    for _, row in team_games_penalty_data.iterrows():
        start_idx = np.searchsorted(time_points, row['Start_Time'])
        end_idx = np.searchsorted(time_points, row['End_Time'])
        if row['Team'] == team:
            team_penalty_count[start_idx:end_idx] += 1
        else:
            opponent_penalty_count[start_idx:end_idx] += 1

    
    

    # Use the color mapping dictionary
    color = color_mapping.get(team, 'default_color')  # Replace 'default_color' with a default color hex

    # Plot the data on the provided ax subplot
    ax.hist(time_points, bins=65, range=[0, 65], weights=team_penalty_count, 
            color=color, alpha=0.9, label=f'{team} Penalties', linewidth=0.5, edgecolor='black')
    ax.hist(time_points, bins=65, range=[0, 65], weights=opponent_penalty_count, color='grey', 
            alpha=0.5, label='Opponent Penalties', linewidth=0.75, edgecolor='black')

    # Customize the subplot as needed
    ax.set_xlabel('')
    ax.set_ylabel(f'{team}', fontsize=14)
    ax.tick_params(axis='y', which='major', length=0)
    ax.tick_params(axis='y', which='major', labelsize=0)
    ax.set_title(f'Penalty Box Occupancy Over Time for {team}')
    ax.set_title('') # REMOVE TITLE IF NOT NEEDED
    ax.grid(True)
    
    # Custom x-axis ticks and labels for the periods
    period_ticks = [10, 30, 50, 62.5]  # Midpoints of 1st, 2nd, 3rd periods and OT
    period_labels = ['1st Period', '2nd Period', '3rd Period', 'OT']
    ax.set_xticks(period_ticks)
    ax.set_xticklabels(period_labels)

    
    # Set grid, background color, and border properties
    ax.grid(True, axis='y', color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.9, zorder=0)
    ax.grid(True, axis='x', color='lightgrey', linestyle='-', linewidth=0.5, alpha=0.9, zorder=0)
    # shade the periods - first and third periods (0 to 20 and 40 to 60) in light grey
    ax.axvspan(0, 20, facecolor='lightgrey', alpha=0.5, zorder=0)
    ax.axvspan(40, 60, facecolor='lightgrey', alpha=0.5, zorder=0)
    # Shade 2nd period and OT in light yeallow
    ax.axvspan(20, 40, facecolor='lightyellow', alpha=0.75, zorder=0)
    ax.axvspan(60, 65, facecolor='lightyellow', alpha=0.75, zorder=0)
    ax.set_axisbelow(True)


def plot_region_penalties(region_name, region_dict, penalty_data, color_mapping, figure_size=figure_size):
    teams_to_plot = final_four_teams
    
    # Determine the maximum y-value across all histograms for the teams in the region
    max_y_value = 0
    for team in teams_to_plot:
        team_penalties = penalty_summary_df[penalty_summary_df['Team'] == team]
        counts, _ = np.histogram(team_penalties['Pen_Length'], bins=65, range=[0, 65])
        max_y_value = max(max_y_value, counts.max())

    # Create a figure with subplots for each team
    fig, axes = plt.subplots(len(teams_to_plot), 1, figsize=figure_size, sharex=True)

    # If there's only one team to plot, axes will not be an array, so we make it one for consistency
    if len(teams_to_plot) == 1:
        axes = [axes]

    # Ensure that the time_points and bins are consistent and aligned across all histograms
    time_points = np.linspace(0, 65, 65 * 60)  # 1-second intervals
    bins = np.linspace(0, 65, 66)  # 65 bins from 0 to 65

    # Plot the penalties for each team using consistent bins and time_points
    for i, team in enumerate(teams_to_plot):
        plot_team_penalties(team, penalty_data, color_mapping, ax=axes[i], bins=bins)
        axes[i].set_ylim([0, max_y_value + 600])  # Use max_y_value + 1 to ensure the highest bin is fully visible

    
    # Set Title for the entire figure
    fig.suptitle(f'Penalty Box Occupancy Over Game Time: 2024 Frozen Four', **font_title_param)

    # set the x-axis label but just for the last plot
    axes[-1].set_xlabel('Team Color: Players In Box | Gray: Opponent In Box | Each Bar is Equal to 1 Minute Increment', **font_label_param)

    # set the x axis labels for the first plot only to label periods
    # axes[-1].set_xticklabels(['1st Period', '2nd Period', '3rd Period', 'OT'], fontsize=12)
    
    # Make the same label but place it on tip of the first plot
    axes[0].tick_params(axis='x', which='major', pad=10, top=True, labeltop=True)

    # Set background to white
    fig.patch.set_facecolor('white')    
    # Remove tick marks from bottom of plot
    # axes[0].tick_params(axis='x', which='major', bottom=False, labelbottom=False)
    # axes[0].set_xlabel('Game Time', font_label_param)
    

    # Adjust the layout and save/show the plot as needed
    plt.tight_layout()\
    
    


    # Save the plot to TEMP_FOLDER
    plot_path = f'{TEMP_FOLDER}{region_name}_penalties_plot.png'
    plt.savefig(plot_path, bbox_inches='tight', transparent=False)


    
    plt.show()


## Output

In [ ]:
## Example Usage
region_name = set_region

# Call the function with the specified parameters\
plot_region_penalties(region_name, final_four_teams, penalty_summary_df, color_mapping)


# Special Teams Scatter Plot

## Code

In [ ]:
# Load Penalty Date (Taken Straight from CHN stats table)
chn_table_path = '../../NCAA/data/CHN_Stats_Table_April_5.csv'
chn_stats = pd.read_csv(chn_table_path)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Load Penalty Data
chn_stats = pd.read_csv('../../NCAA/data/CHN_Stats_Table_April_5.csv')

# Filter DataFrame for just final four teams and select the columns we want
final_four_teams = ['Boston College', 'Michigan', 'Boston University', 'Denver']
chn_stats = chn_stats[chn_stats['Team'].isin(final_four_teams)]

# Setup figure and axes
fig, ax = plt.subplots(figsize=figure_size, dpi=600)

# Scatter and regression plots
sns.scatterplot(data=chn_stats, x='PK%', y='PP%', color='grey', ax=ax)
sns.regplot(data=chn_stats, x='PK%', y='PP%', scatter=False, color='blue', ax=ax)

# Calculate new x and y limits to accommodate annotations and logos
xmin, xmax = ax.get_xlim()
ymin, ymax = ax.get_ylim()
new_xmax = xmax + 2  # Extra space for logos
new_ymax = ymax + 1  # Extra space for title
ax.set_xlim(xmin, new_xmax)
ax.set_ylim(ymin, new_ymax)

# Draw the trend line over the new extended x-axis range
pk_percent_extended = np.linspace(xmin, new_xmax, 500)
pp_percent_trend_extended = np.poly1d(np.polyfit(chn_stats['PK%'], chn_stats['PP%'], 1))(pk_percent_extended)
ax.plot(pk_percent_extended, pp_percent_trend_extended, color='blue', linestyle='--', linewidth=0.8)

# Draw average lines and error bars
ax.axhline(y=chn_stats['PP%'].mean(), color='red', linestyle='--', linewidth=0.8)
ax.axvline(x=chn_stats['PK%'].mean(), color='red', linestyle='--', linewidth=0.8)
ax.errorbar(chn_stats['PK%'].mean(), chn_stats['PP%'].mean(), xerr=chn_stats['PK%'].std(), yerr=chn_stats['PP%'].std(), color='purple', fmt='o')

# Set axis labels and title
ax.set_title('Special Teams: 2024 Frozen Four', fontsize=20)
ax.set_xlabel('Penalty Kill %', fontsize=14)
ax.set_ylabel('Power Play %', fontsize=14)
ax.set_facecolor('white')

# Overlay logos for specific teams in the region
teams_in_region = final_four_teams
add_logos_to_plot(chn_stats, 'PK%', 'PP%', logo_mapping, logo_folder, ax, teams_in_region)

# Add a table at the top right of the plot & Set Parameters
table = plt.table(cellText=table_data_list, loc='best', 
                    colWidths=col_widths,
                    cellLoc='center', 
                    bbox=[0.45, 0, 0.5, 0.3]
                    )

# Customize the table properties
table.auto_set_font_size(False)
table.set_fontsize(12)  # Set the fontsize
table.scale(1, 1.5)  # Scale the table for width and height

# Set individual cell properties
for (row, col), cell in table.get_celld().items():
    if row == 0:
        # Header row
        cell.set_text_props(weight='bold')  # Make the header bold
        cell.set_facecolor('lightgrey')  # Header background color
    else:
        # All other rows
        cell.set_facecolor('none')  # Make cells transparent
        # cell.set_text_props(weight='bold')  # Optionally make text bold
    cell.set_edgecolor('w')  # Set the border color to white


# Set axis labels and title font parameters
ax.set_title('Special Teams: 2024 Frozen Four', fontdict=font_title_param)
ax.set_xlabel('Penalty Kill %', fontdict=font_label_param)
ax.set_ylabel('Power Play %', fontdict=font_label_param)
# Tick labels font parameters
ax.tick_params(axis='both', labelsize=12)  # Update label size as needed

# # add the same background image as the rest of the plots use
# ax.imshow(bg_img, aspect='auto', extent=[xmin, new_xmax, ymin, new_ymax], zorder=0, alpha=alpha_value)



# Save plot To TEMP_FOLDER
plot_path = 'final_four_special_teams_plot.png'  # Update the path as needed
plt.savefig(plot_path, bbox_inches='tight')

plt.show()



In [ ]:
# Load Penalty Date (Taken Straight from CHN stats table)
chn_table_path = '../../NCAA/data/CHN_Stats_Table_April_5.csv'
chn_stats = pd.read_csv(chn_table_path)


# Filter DataFrame for just final_four_teams and select the columns we want
final_four_teams = ['Boston College', 'Michigan', 'Boston University', 'Denver']  # Define your final four teams
table_data = chn_stats[chn_stats['Team'].isin(final_four_teams)]
table_data = table_data.sort_values(by='Team')  # Order the data using the earlier variable
table_data = table_data[['Team', 'PK%', 'PP%']]  # Select just the columns we want
table_data_list = [table_data.columns.values.tolist()] + table_data.values.tolist()  # Convert to list of lists

# Define column widths
col_widths = [0.2, 0.2, 0.2]  # Adjust these values as necessary

# Assuming you've set figure size, dpi, and font parameters as previously discussed
fig, ax = plt.subplots(figsize=(10, 10), dpi=600)  # Update figsize and dpi as per your setup

# Plot settings
sns.scatterplot(data=chn_stats, x='PK%', y='PP%', color='grey', ax=ax)
sns.regplot(data=chn_stats, x='PK%', y='PP%', scatter=False, color='blue', ax=ax)

# Adjust the upper x limit to provide space for top logo
xmin, xmax = ax.get_xlim()
new_xmax = xmax + 2
ax.set_xlim(xmin, new_xmax)

# Do the same for the y limit
ymin, ymax = ax.get_ylim()
new_ymax = ymax + 1
ax.set_ylim(ymin, new_ymax)

# Draw the background image after setting the new limits
# bg_img = plt.imread('path_to_your_background_image.png')  # make sure to provide the correct path
# alpha_value = 0.1  # adjust your desired transparency
ax.imshow(bg_img, aspect='auto', extent=[xmin, new_xmax, ymin, new_ymax], zorder=0, alpha=alpha_value)


### NEW
# Draw the trend line over the new extended x-axis range
pk_percent_extended = np.linspace(xmin, new_xmax, 500)
pp_percent_trend_extended = np.poly1d(np.polyfit(chn_stats['PK%'], chn_stats['PP%'], 1))(pk_percent_extended)
ax.plot(pk_percent_extended, pp_percent_trend_extended, color='blue', linestyle='--', linewidth=0.8)

# Draw average lines and error bars
ax.axhline(y=chn_stats['PP%'].mean(), color='red', linestyle='--', linewidth=0.8)
ax.axvline(x=chn_stats['PK%'].mean(), color='red', linestyle='--', linewidth=0.8)
ax.errorbar(chn_stats['PK%'].mean(), chn_stats['PP%'].mean(), xerr=chn_stats['PK%'].std(), yerr=chn_stats['PP%'].std(), color='purple', fmt='o')

# # Add trend line, averages, and standard deviations after the new limits
# # Recalculate the pp_percent_trend with the extended x-axis range
# pk_percent_extended = np.linspace(xmin, new_xmax, 500)  # Increase the number of points for a smoother line
# pp_percent_trend_extended = np.poly1d(np.polyfit(chn_stats['PK%'], chn_stats['PP%'], 1))(pk_percent_extended)

# # Update the trend line plot
# ax.plot(pk_percent_extended, pp_percent_trend_extended, color='blue', linestyle='--', linewidth=0.8)

# # pk_percent = np.linspace(xmin, new_xmax, 100)  # Generate 100 points from xmin to new_xmax
# # pp_percent_trend = np.poly1d(np.polyfit(chn_stats['PK%'], chn_stats['PP%'], 1))(pk_percent)
# # ax.plot(pk_percent, pp_percent_trend, color='blue', linestyle='--', linewidth=0.8)

# ax.axhline(y=chn_stats['PP%'].mean(), color='red', linestyle='--', linewidth=0.8)
# ax.axvline(x=chn_stats['PK%'].mean(), color='red', linestyle='--', linewidth=0.8)
# ax.errorbar(chn_stats['PK%'].mean(), chn_stats['PP%'].mean(), xerr=chn_stats['PK%'].std(), yerr=chn_stats['PP%'].std(), color='purple', fmt='o')


# Titles and Labels with font parameters
ax.set_title('Special Teams: 2024 Frozen Four', fontsize=20)  # Update font size as needed
ax.set_xlabel('Penalty Kill %', fontsize=14)  # Update font size as needed
ax.set_ylabel('Power Play %', fontsize=14)  # Update font size as needed

ax.set_facecolor('white')  # Assuming you want a white background

# Adjust tick labels with font parameters
ax.tick_params(axis='both', labelsize=12)  # Update label size as needed

# Overlay logos for specific teams in the region
teams_in_region = final_four_teams
add_logos_to_plot(chn_stats, 'PK%', 'PP%', logo_mapping, logo_folder, ax, teams_in_region)

# Add a table at the top right of the plot & Set Parameters
table = plt.table(cellText=table_data_list, loc='best', 
                    colWidths=col_widths,
                    cellLoc='center', 
                    bbox=[0.45, 0, 0.5, 0.3]
                    )

# Customize the table properties
table.auto_set_font_size(False)
table.set_fontsize(12)  # Set the fontsize
table.scale(1, 1.5)  # Scale the table for width and height

# Set individual cell properties
for (row, col), cell in table.get_celld().items():
    if row == 0:
        # Header row
        cell.set_text_props(weight='bold')  # Make the header bold
        cell.set_facecolor('lightgrey')  # Header background color
    else:
        # All other rows
        cell.set_facecolor('none')  # Make cells transparent
        # cell.set_text_props(weight='bold')  # Optionally make text bold
    cell.set_edgecolor('w')  # Set the border color to white


# Set axis labels and title font parameters
ax.set_title('Special Teams: 2024 Frozen Four', fontdict=font_title_param)
ax.set_xlabel('Penalty Kill %', fontdict=font_label_param)
ax.set_ylabel('Power Play %', fontdict=font_label_param)
# Tick labels font parameters
ax.tick_params(axis='both', labelsize=12)  # Update label size as needed

# # add the same background image as the rest of the plots use
# ax.imshow(bg_img, aspect='auto', extent=[xmin, new_xmax, ymin, new_ymax], zorder=0, alpha=alpha_value)



# Save plot To TEMP_FOLDER
plot_path = 'final_four_special_teams_plot.png'  # Update the path as needed
plt.savefig(plot_path, bbox_inches='tight')

plt.show()



In [ ]:
# Load Penalty Date (Taken Straight from CHN stats table)
chn_table_path = '../../NCAA/data/CHN_Stats_Table_April_5.csv'
chn_stats = pd.read_csv(chn_table_path)

# Create the data to display in the table
# Filter df for just final_four_teams
table_data = chn_stats[chn_stats['Team'].isin(final_four_teams)]
# order the data usign the earlier variable
table_data = table_data.sort_values(by='Team')
# select just he columns we want
table_data = table_data[['Team', 'PK%', 'PP%']]
# convert to list of lists
table_data_list = [table_data.columns.values.tolist()] + table_data.values.tolist()


# Assuming you've set figure size, dpi, and font parameters as previously discussed
fig, ax = plt.subplots(figsize=figure_size, dpi=dpi)

# Plot settings
sns.scatterplot(data=chn_stats, x='PK%', y='PP%', color='grey', ax=ax)
sns.regplot(data=chn_stats, x='PK%', y='PP%', scatter=False, color='blue', ax=ax)

# Add trend line, averages, and standard deviations as before
pk_percent = np.unique(chn_stats['PK%'])
pp_percent_trend = np.poly1d(np.polyfit(chn_stats['PK%'], chn_stats['PP%'], 1))(pk_percent)
ax.plot(pk_percent, pp_percent_trend, color='blue', linestyle='--', linewidth=0.8)

ax.axhline(y=chn_stats['PP%'].mean(), color='red', linestyle='--', linewidth=0.8)
ax.axvline(x=chn_stats['PK%'].mean(), color='red', linestyle='--', linewidth=0.8)
ax.errorbar(chn_stats['PK%'].mean(), chn_stats['PP%'].mean(), xerr=chn_stats['PK%'].std(), yerr=chn_stats['PP%'].std(), color='purple', fmt='o')

# Assuming add_logos_to_plot function is adapted for the new dataset
# add_logos_to_plot(chn_stats, 'PK%', 'PP%', team_colors, logo_mapping, ax=ax)

# Titles and Labels with font parameters
ax.set_title('Special Teams: 2024 Frozen Four', **font_title_param)
ax.set_xlabel('Penalty Kill %', **font_label_param)
ax.set_ylabel('Power Play %', **font_label_param)

ax.set_facecolor('white')  # Assuming you want a white background

# Adjust tick labels with font parameters
# Adjusting tick label sizes
ax.tick_params(axis='both', labelsize=size3)

# Add a table at the top right of the plot & Set Parameters
table = plt.table(cellText=table_data_list, loc='upper center',
                  colWidths=col_widths,
                  cellLoc='center', 
                  bbox=[0, 1.05, 1, 0.1])  # This bbox places the table above the plot


# Optionally setting font family for tick labels
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontfamily(font)
    label.set_fontsize(size3)  # This is redundant if size3 was used in tick_params
    # Add a table at the top right of the plot & Set Parameters
table = plt.table(cellText=table_data, loc='best', 
                    colWidths=col_widths,
                    cellLoc='center', 
                    bbox=[0.45, 0, 0.5, 0.3]
                    )
    
# Customize the table properties
table.auto_set_font_size(False)
table.set_fontsize(12)  # Set the fontsize
table.scale(1, 1.5)  # Scale the table for width and height

# Set individual cell properties
for (row, col), cell in table.get_celld().items():
    if row == 0:
        # Header row
        cell.set_text_props(weight='bold')  # Make the header bold
        cell.set_facecolor('lightgrey')  # Header background color
    else:
        # All other rows
        cell.set_facecolor('none')  # Make cells transparent
        # cell.set_text_props(weight='bold')  # Optionally make text bold
    cell.set_edgecolor('w')  # Set the border color to white

# Add title and axis labels
plt.title(f'Goals Per Game: {region_name} Region', fontdict=font_title_param)
plt.xlabel('Average Goals Allowed', fontdict=font_label_large_param)
plt.ylabel('Average Goals Scored', fontdict=font_label_large_param)

# Save plot To TEMP_FOLDER
plot_path = f'{TEMP_FOLDER}final_four_goals_plot.png'

# Save
# plt.savefig(plot_path, bbox_inches='tight')


plt.show()


# Physical Comparison Chart

## Code

In [ ]:
final_four_teams

In [ ]:
# Check master roster by getting value counts of Team
# Pull Master_roster from sql
master_roster_query = """
SELECT * FROM master_roster;
"""

# Read the master roster table into a dataframe
master_roster_df = pd.read_sql(master_roster_query, conn)

# Check the value counts of the 'Team' column
master_roster_df['Team'].value_counts()

In [ ]:

# Assuming final_four_teams is a list of team names
final_four_teams_str = ', '.join(f"'{team}'" for team in final_four_teams)

roster_query_filtered = f"""
SELECT * 
FROM master_roster 
WHERE REPLACE(Team, '-', ' ') IN ({final_four_teams_str});
"""

roster_df_filtered = pd.read_sql(roster_query_filtered, conn)
# Remove the - from the team names
roster_df_filtered['Team'] = roster_df_filtered['Team'].str.replace('-', ' ')
# Check Team Value Counts
roster_df_filtered['Team'].value_counts()


In [ ]:

# Set the order of the positions
position_order = ['Forwards', 'Defensemen', 'Goaltenders']

# Set the order of Class Rank
class_order = ['Fr', 'So', 'Jr', 'Sr', 'Gr']

# Set the Team order
team_order = ['Boston College', 'Michigan', 'Boston University', 'Denver']


# # # # Create a filtered dataframe for Schools We want to highlight from the master roster in the database
# # roster_query_filtered = "SELECT * FROM master_roster WHERE Team IN ('Boston College', 'Michigan', 'Boston University', 'Denver');"

# # # Get the entire roster from the master roster in the database
# # roster_query_full = f"SELECT * FROM master_roster;"

# df_filtered = pd.read_sql(roster_query_filtered, conn)
# df_full = pd.read_sql(roster_query_full, conn)

df_filtered = roster_df_filtered
df_full = master_roster_df

# Filter out any rows from df_full where Wt or Height is null or 0
df_full = df_full[(df_full['Wt'].notnull()) & (df_full['Height_Inches'].notnull())]
df_full = df_full[(df_full['Wt'] != 0) & (df_full['Height_Inches'] != 0)]



########### Create the Plots ###########
def generate_plot(x_col, y_col, title, x_order, y_label, df, df_filtered, team_colors, bg_img, figure_size, font_title_param, font_label_param, font_tick_param, TEMP_FOLDER):
    
    # Create subplot
    fig, ax = plt.subplots(1, 1, figsize=figure_size, dpi=dpi)
    
    # Set Axis ranges
    ax.set_xlim([0, len(x_order)])

    # Background Image
    x_range = ax.get_xlim()
    y_min = df[y_col].min()
    y_max = df[y_col].max()
    ax.imshow(bg_img, aspect='auto', extent=[((x_range[0])-1), x_range[1], y_min, y_max], alpha=0.1, zorder=0)

    # Plots
    sns.boxplot(x=x_col, y=y_col, data=df, color='lightgrey', showfliers=False, ax=ax, order=x_order, zorder=1)
    # sns.swarmplot(x=x_col, y=y_col, data=df, color='grey', alpha=0.5, ax=ax, order=x_order)
    ## Make a strip plot with jitter and a smaller point size
    sns.stripplot(x=x_col, y=y_col, data=df, color='grey', alpha=0.5, ax=ax, order=x_order, size=4, jitter=0.2, zorder=2)
    sns.boxplot(x=x_col, y=y_col, data=df_filtered, hue='Team', ax=ax, 
                palette=team_colors, showfliers=False, order=x_order,
                width=0.65, linewidth=2, hue_order=team_order)


    # Titles and Labels
    ax.set_title(title, fontdict=font_title_param)
    ax.set_ylabel(y_label, fontdict=font_label_param)
    ax.set_xticklabels(ax.get_xticklabels(), fontdict=font_tick_param)
    ax.set_yticklabels(ax.get_yticklabels(), fontdict=font_tick_param)
    ax.set_xlabel('')

    ########## Conver the tick marks to feet and inches in the Height Plots ##########
    # Check if y_col is for height and then apply the conversion
    if y_col == 'Height_Inches':
        ax.set_yticks(ax.get_yticks())
        ax.set_yticklabels([f'{int(tick // 12)}\'{int(tick % 12)}"' for tick in ax.get_yticks()], fontdict=font_tick_param)
    else:
        ax.set_yticklabels(ax.get_yticklabels(), fontdict=font_tick_param)

    
    # Get upper bounds of the axis
    x_max = ax.get_xlim()[1]
    y_max = ax.get_ylim()[1]

    # Get lower bounds of the axis
    x_min = ax.get_xlim()[0]
    y_min = ax.get_ylim()[0]

    # # Define a function to place logos at given coordinates
    # def place_logo(image_path, xy, ax, zoom=0.3):
    #     img = plt.imread(image_path)
    #     imagebox = OffsetImage(img, zoom=zoom)
    #     ab = AnnotationBbox(imagebox, xy, frameon=False, pad=0, xycoords='axes fraction', boxcoords="axes fraction")
    #     ax.add_artist(ab)
    
##########################################
    # # Place logos at upper corners
    # place_logo(logo_folder + team_1_logo, (0.80, 0.90), ax)  # Upper left corner
    # place_logo(logo_folder + team_2_logo, (0.20, 0.90), ax)  # Upper right corner
############################################

    # # # Drop Legend if there is any
    if ax.get_legend() is not None:
        ax.get_legend().remove()
        # # # Tight Layout
    plt.tight_layout()
    
    # Assuming 'team_colors' dictionary is correctly populated with your teams and their respective colors
    handles = [plt.Rectangle((0,0),1,1, color=team_colors[team]) for team in team_order]
    labels = team_order

    # Adding legend to the figure, not the axes (to position it relative to the entire figure)
    legend = fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.05), ncol=len(team_order), fontsize=12)

    # Adjust layout to make space for the legend
    plt.subplots_adjust(bottom=0.05)  # You may need to adjust this value based on your figure's size



    save_path = f'{TEMP_FOLDER}'
    os.makedirs(save_path, exist_ok=True)  # Create directory if doesn't exist
    
    # Save Plot
    plt.savefig(f'{save_path}{title}.png', bbox_inches='tight', dpi=600)
    
    plt.show()


## Output

In [ ]:

# Weight by Class Rank:
# generate_plot('Yr.', 'Wt.', 'Weight by Class Year', ['Fr', 'So', 'Jr', 'Sr', 'Gr'], 'LBs', df_current, df_filtered, team_colors, bg_img, figure_size, font_title_param, font_label_param, font_tick_param, TEMP_FOLDER)

# Height by Class Rank:
# generate_plot('Yr.', 'Height_Inches', 'Height by Class Year', ['Fr', 'So', 'Jr', 'Sr', 'Gr'], 'Height', df_current, df_filtered, team_colors, bg_img, figure_size, font_title_param, font_label_param, font_tick_param, TEMP_FOLDER)

# Assuming df_full is the full dataset and df_filtered is already filtered for the final four teams
generate_plot('Position', 'Wt', 'Weight by Position: 2024 Frozen Four', position_order, 
              'Weight (lb)', df_full, df_filtered, color_mapping, 
              bg_img, figure_size, font_title_param, font_label_param, 
              font_tick_param, TEMP_FOLDER)

generate_plot('Position', 'Height_Inches', 'Height by Position: 2024 Frozen Four', position_order,
               'Height', df_full, df_filtered, color_mapping, bg_img,
                 figure_size, font_title_param, font_label_param,
                   font_tick_param, TEMP_FOLDER)




# Line By Line Stats

## Code

In [ ]:
away_games_df = pd.read_sql("""
WITH UniqueGames AS (
    SELECT DISTINCT * FROM game_details
)
SELECT 
    Away_Team AS Team, 
    COUNT(DISTINCT Game_ID) AS Num_Games_Away
FROM UniqueGames
GROUP BY Away_Team;
""", conn)

home_games_df = pd.read_sql("""
WITH UniqueGames AS (
    SELECT DISTINCT * FROM game_details
)
SELECT 
    Home_Team AS Team, 
    COUNT(DISTINCT Game_ID) AS Num_Games_Home
FROM UniqueGames
GROUP BY Home_Team;
""", conn)



games_played_count = pd.merge(away_games_df, home_games_df, how='outer', on='Team')
games_played_count['Num_Games'] = games_played_count['Num_Games_Away'].fillna(0) + games_played_count['Num_Games_Home'].fillna(0)
games_played_count = games_played_count[['Team', 'Num_Games']]

# games_played_count.tail()

In [ ]:
# Query to get line-specific stats for goals, assists, and penalty minutes scored for each team,
# now also including the Position column to distinguish between Forwards and Defensemen
line_goals_assists_pim_position_query = """
WITH UniqueLineChart AS (
    SELECT DISTINCT Team, Game_ID, Line, Player, Position
    FROM line_chart
    WHERE Position != 'Goalie'
),
UniquePlayerStats AS (
    SELECT DISTINCT Game_ID, Team, Player, G, A, Sh, plus_minus, PIM
    FROM player_stats
)
SELECT
    ulc.Team,
    ulc.Line,
    ulc.Position,
    SUM(ups.G) AS Total_Goals,
    SUM(ups.A) AS Total_Assists,
    SUM(ups.Sh) AS Total_Shots,
    SUM(ups.G) + SUM(ups.A) AS Total_Points,
    SUM(ups.plus_minus) AS Total_Plus_Minus,
    SUM(ups.PIM) AS Total_PIM
FROM UniqueLineChart AS ulc
LEFT JOIN UniquePlayerStats AS ups ON ulc.Game_ID = ups.Game_ID AND ulc.Team = ups.Team AND ulc.Player = ups.Player
GROUP BY ulc.Team, ulc.Line, ulc.Position
ORDER BY ulc.Team, ulc.Line, ulc.Position;

"""

# Execute the query and store the results in a DataFrame
line_goals_assists_pim_position_df = pd.read_sql(line_goals_assists_pim_position_query, conn)

# Create a new column 'Type' to distinguish between Forwards and Defensemen
line_goals_assists_pim_position_df['Type'] = line_goals_assists_pim_position_df['Position'].apply(lambda x: 'Forward' if x in ['Left Wing', 'Right Wing', 'Center'] else 'Defense')

# Step 1: Create 'Type' column
# (You've already done this part)
line_goals_assists_pim_position_df['Type'] = line_goals_assists_pim_position_df['Position'].apply(
    lambda x: 'Forward' if x in ['Left Wing', 'Right Wing', 'Center'] else 'Defense'
)

# Step 2: Aggregate again by 'Type'
agg_columns = ['Total_Goals', 'Total_Assists', 'Total_PIM', 'Total_Points', 'Total_Shots', 'Total_Plus_Minus']
grouped_df = line_goals_assists_pim_position_df.groupby(['Team', 'Line', 'Type'])[agg_columns].sum().reset_index()

# Now, you can merge the 'Num_Games' and calculate the averages as before
grouped_df = pd.merge(grouped_df, games_played_count, how='left', on='Team')

# Calculate the average stats per game
grouped_df['AVG_Goals'] = grouped_df['Total_Goals'] / grouped_df['Num_Games']
grouped_df['AVG_Assists'] = grouped_df['Total_Assists'] / grouped_df['Num_Games']
grouped_df['AVG_Points'] = grouped_df['Total_Points'] / grouped_df['Num_Games']
grouped_df['AVG_PIM'] = grouped_df['Total_PIM'] / grouped_df['Num_Games']
grouped_df['AVG_Shots'] = grouped_df['Total_Shots'] / grouped_df['Num_Games']
grouped_df['AVG_Plus_Minus'] = grouped_df['Total_Plus_Minus'] / grouped_df['Num_Games']


# Rename grouped_df to line_goals_assists_pim_position_df
line_goals_assists_pim_position_df = grouped_df

In [ ]:
data = line_goals_assists_pim_position_df

# Filter data to just teams in the tourney
# pairwise_df Teams to a list
tourny_list = pairwise_df['Team'].to_list()
tourney_team_data = data[data['Team'].isin(tourny_list)]

def plot_data(data, stat, ax, team_colors, team_order, set_linewidth, data_type='Forward'):
    # Determine if we need to filter out line 4
    if data_type == 'Defense':
        # Ensure to exclude line 4 from the background data for defense plots
        background_data = tourney_team_data[(tourney_team_data['Type'] == 'Defense') & (tourney_team_data['Line'] != '4')]
    else:
        # Use all data for forwards
        background_data = tourney_team_data[tourney_team_data['Type'] == 'Forward']

    # Plot the background bars for the filtered teams
    sns.barplot(x='Line', y=stat, hue='Team', data=background_data, 
                color='lightgrey', alpha=0.6, ax=ax, ci=None)

    # Plot the foreground bars for the specified teams
    filtered_data = data[data['Team'].isin(team_order)]
    sns.barplot(x='Line', y=stat, hue='Team', data=filtered_data, 
                palette=team_colors, ax=ax, ci=None, alpha=0.6, edgecolor='black', linewidth=set_linewidth, hue_order=team_order)
    
    # Remove the legend
    ax.get_legend().remove()


def add_background_shading(ax, x_breaks, colors):
    for i, color in enumerate(colors):
        ax.axvspan(x_breaks[i][0], x_breaks[i][1], facecolor=color, alpha=0.5, zorder=0)

def plot_region_stats(region_name, region_dict, data, team_colors, set_linewidth, all_color, figure_size=(10, 10), dpi=100):
    # Define the figure size and create subplots
    fig, axs = plt.subplots(4, 2, figsize=figure_size, dpi=dpi)

    # Filter the data for teams in the region
    teams_to_plot = final_four_teams
    filtered_data = data[data['Team'].isin(teams_to_plot)]

    # Define stats
    stats = ['AVG_Goals', 'AVG_Shots', 'AVG_PIM', 'AVG_Plus_Minus']

    forwards_data = filtered_data[filtered_data['Type'] == 'Forward']
    defense_data = filtered_data[filtered_data['Type'] == 'Defense']
    defense_data = defense_data[defense_data['Line'] != '4']


    # Plotting logic
    for i, stat in enumerate(stats):
        plot_data(forwards_data, stat, axs[i, 0], team_colors, teams_to_plot, set_linewidth, data_type='Forward')
        plot_data(defense_data, stat, axs[i, 1], team_colors, teams_to_plot, set_linewidth, data_type='Defense')
   
            
        
        
        # Background shading
        forward_x_breaks = [(0.5, 1.5), (1.5, 2.5), (2.5, 3.5), (3.5, 4.5)]
        defense_x_breaks = [(0.5, 1.5), (1.5, 2.5)]
        colors = ['lightgrey', 'lightyellow']
        add_background_shading(axs[i, 0], forward_x_breaks, colors)
        add_background_shading(axs[i, 1], defense_x_breaks, colors)

        # Changle Axis Titles and Style for the plots
        axs[0, 0].set_ylabel(f'Goals', **font_title_param)
        axs[1, 0].set_ylabel(f'Shots', **font_title_param)
        axs[2, 0].set_ylabel(f'PIM', **font_title_param)
        axs[3, 0].set_ylabel(f'+ / -', **font_title_param)
        # Remove Y Labels Defence plots
        axs[0, 1].set_ylabel('')
        axs[1, 1].set_ylabel('')
        axs[2,1].set_ylabel('')
        axs[3,1].set_ylabel('')

        # Set x tick marks to off on all plots
        axs[0, 0].tick_params(axis='x', which='major', length=0)
        axs[0, 1].tick_params(axis='x', which='major', length=0)
        axs[1, 0].tick_params(axis='x', which='major', length=0)
        axs[1, 1].tick_params(axis='x', which='major', length=0)
        axs[2,0].tick_params(axis='x', which='major', length=0)
        axs[2,1].tick_params(axis='x', which='major', length=0)

        # Remove x-axis tick labels and marks for all but the bottom plots
        axs[0, 0].set_xticklabels([])
        axs[1, 0].set_xticklabels([])
        axs[2, 0].set_xticklabels([])
        axs[0, 1].set_xticklabels([])
        axs[1, 1].set_xticklabels([])
        axs[2, 1].set_xticklabels([])

        # Remove x labels from all but the bottom plots
        axs[0, 0].set_xlabel('')
        axs[1, 0].set_xlabel('')
        axs[2, 0].set_xlabel('')
        axs[0, 1].set_xlabel('')
        axs[1, 1].set_xlabel('')
        axs[2, 1].set_xlabel('')
        # Set the x-axis label for the bottom plots
        axs[2, 0].set_xlabel('Line', **font_label_large_param)
        axs[2, 1].set_xlabel('Line', **font_label_large_param)
        # Set the x-axis label for the bottom plots
        axs[3, 0].set_xlabel('')
        axs[3, 1].set_xlabel('')
        

        # Set The tile for just the top 2 plots
        axs[0, 0].set_title('Forwards', **font_label_large_param)
        axs[0, 1].set_title('Defense', **font_label_large_param)

        # Set title for the entire figure
        fig.suptitle(f'Average per Game by Line: 2024 Frozen Four', **font_title_param)

        # Set layout to tight
        # fig.tight_layout()

        # Set background to white
        fig.patch.set_facecolor('white')    

        # Apply tight layout first without the bottom space for the legend
        fig.tight_layout(rect=[0, 0.02, 1, 0.98])

        # Then adjust subplots to leave space for the legend at the bottom
        fig.subplots_adjust(bottom=0.1)  # You may need to tweak this value

        # After all plotting and customizations are done, but before plt.show()

        # Set hardcoded y-axis lower limit for the bottom row plots (Plus Minus stats)
        lower_y_limit = -1  # Example value; adjust as needed
        axs[3, 0].set_ylim(bottom=lower_y_limit)
        axs[3, 1].set_ylim(bottom=lower_y_limit)

        # Get the upper y limit from the data
        upper_y_limit = max(data['AVG_Plus_Minus'].max(), data['AVG_Plus_Minus'].max())
        axs[3, 0].set_ylim(top=upper_y_limit + 0.2)
        axs[3, 1].set_ylim(top=upper_y_limit + 0.2)

        
        
        # Create a custom legend with team names to the very bottom of the figure
        handles = [plt.Rectangle((0, 0), 1, 1, color=team_colors[team]) for team in teams_to_plot]
        labels = teams_to_plot
        # add the legend at the bottom
        legend = fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 0), ncol=len(teams_to_plot), fontsize=12)
        plt.subplots_adjust(bottom=0.1)  # Adjust this if necessary to make room for legend

        legend

    

        # Save plot to TEMP_FOLDER
        plot_path = f'{TEMP_FOLDER}{region_name}_line_by_line.png'
        plt.savefig(plot_path, bbox_inches='tight')
    
    # Show the plot
    plt.show()


## Output

In [ ]:
# Corrected function call
plot_region_stats(
    region_name=set_region,
    region_dict=final_four_teams,
    data=line_goals_assists_pim_position_df,
    team_colors=team_colors_dict,
    set_linewidth=2,
    all_color='#CCCCCC',  # default color for background data
    figure_size=figure_size,
    dpi=100
)